In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden GenAI Workshop for Flux

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_flux_gradio.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_flux_gradio.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates starting a playground for [black-forest-labs/FLUX.1-schnell](https://huggingface.co/black-forest-labs/FLUX.1-schnell) model based on [Gradio UI](https://www.gradio.app/), which allows users to interact with the identity-preserving image generation model more easily and intuitively.

### Objective

- Deploy model to a [Vertex AI Endpoint resource](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for `text-to-image` tasks from the UI.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Run the playground

In [ ]:
# @title Setup Google Cloud project and prepare dependencies

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

# @markdown 3. If you want to run predictions with A100 80GB or H100 GPUs, we recommend using the regions listed below. **NOTE:** Make sure you have associated quota in selected regions. Click the links to see your current quota for each GPU type: [Nvidia A100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_a100_80gb_gpus), [Nvidia H100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_h100_gpus). You can request for quota following the instructions at ["Request a higher quota"](https://cloud.google.com/docs/quota/view-manage#requesting_higher_quota).

# @markdown | Machine Type | Accelerator Type | Recommended Regions |
# @markdown | ----------- | ----------- | ----------- |
# @markdown | a2-ultragpu-1g | 1 NVIDIA_A100_80GB | us-central1, us-east4, europe-west4, asia-southeast1, us-east4 |
# @markdown | a3-highgpu-2g | 2 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-4g | 4 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-8g | 8 NVIDIA_H100_80GB | us-central1, europe-west4, us-west1, asia-southeast1 |

! pip3 install --upgrade gradio==4.29.0 opencv-python
# Uninstall nest-asyncio and uvloop as a workaround to https://github.com/gradio-app/gradio/issues/8238#issuecomment-2101066984
! pip3 uninstall --yes nest-asyncio uvloop
# A workaround for the compatibility between the fastapi and pydantic
! pip3 install fastapi==0.112.3

import importlib
import os

from google.cloud import aiplatform

! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)

! gcloud config set project $PROJECT_ID

In [ ]:
# @title Start the playground

# @markdown This is a playground for running Flux models.

# @markdown After running the cell, a public URL (["https://*.gradio.live"](#)) will appear in the cell output. The playground is available in a separate browser tab when you click the URL.

# @markdown **How to use:**
# @markdown 1. Select or deploy model
# @markdown   1. In the playground, select a previous deployed Flux model.
# @markdown   2. If you don't have any deployed model, deploy a new model in the playground.
# @markdown   3. New deployment takes ~20 minutes. You can check the progress at [Vertex Online Prediction](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).
# @markdown   4. After the model deployment is complete, restart the playground in Colab to see the updated endpoint list.
# @markdown 1. Inference
# @markdown   1. In the "Inference" section, fill in prompt and parameters
# @markdown   2. Click "Generate" to generate image from text prompt.

# @markdown **Important notes**
# @markdown 1. Reruning this notebook cell creates a new public URL. Previous URLs will stop working.
# @markdown 2. After experiments, manually undeploy models to avoid continuous charges to the project.
# @markdown 3. This playground is specially built for the Flux models.
# @markdown Other models may work, but they are not tested. Use with caution.

import gradio as gr
from google.cloud import aiplatform
from PIL import Image

# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/pytorch-inference.cu125.0-4.ubuntu2204.py310"


def is_flux_endpoint(endpoint: aiplatform.Endpoint) -> bool:
    """Returns True if the endpoint is an Flux endpoint."""
    return "flux" in endpoint.display_name.lower()


def list_endpoints() -> list[str]:
    """Returns all valid prediction endpoints for in the project and region."""
    # Gets all the valid endpoints in the project and region.
    endpoints = aiplatform.Endpoint.list(order_by="create_time desc")
    # Filters out the endpoints which do not have a deployed model, and the endpoint is for image generation
    endpoints = list(
        filter(
            lambda endpoint: endpoint.traffic_split and is_flux_endpoint(endpoint),
            endpoints,
        )
    )

    endpoint_names = list(
        map(
            lambda endpoint: f"{endpoint.name} - {endpoint.display_name[:40]}",
            endpoints,
        )
    )

    return endpoint_names


def get_endpoint(endpoint_name: str) -> aiplatform.Endpoint:
    """Returns a Vertex endpoint for the given endpoint_name."""

    endpoint_id = endpoint_name.split(" - ")[0]
    endpoint = aiplatform.Endpoint(
        f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_id}"
    )

    return endpoint


def deploy_model(model_id: str, accelerator_type: str) -> aiplatform.Endpoint:
    """Creates a new Vertex prediction endpoint and deploys a model to it."""

    if not model_id:
        raise gr.Error("Select a valid model name for model list.")
        return

    gr.Info("Model deployment started.")

    task_id = "text-to-image"
    display_name = common_util.create_job_name(model_id)
    endpoint = aiplatform.Endpoint.create(display_name=display_name)
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task_id,
        "DEPLOY_SOURCE": "notebook_gradio",
    }

    machine_type_map = {
        "NVIDIA_TESLA_A100": "a2-highgpu-1g",
        "NVIDIA_A100_80GB": "a2-ultragpu-1g",
    }
    machine_type = machine_type_map.get(accelerator_type)
    accelerator_count = 1
    common_util.check_quota(
        project_id=PROJECT_ID,
        region=REGION,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        is_for_training=False,
    )

    display_name = common_util.create_job_name(model_id)
    model = aiplatform.Model.upload(
        display_name=model_id,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predict",
        serving_container_health_route="/health",
        serving_container_environment_variables=serving_env,
        model_garden_source_model_name="publishers/black-forest-labs/models/flux1-schnell",
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
        sync=False,
        system_labels={"NOTEBOOK_NAME": "model_garden_pytorch_flux_gradio.ipynb"},
    )

    gr.Info(
        f"Model {display_name} is being deployed. It may take ~20 minutes to complete."
    )

    return endpoint


def get_default_image_height(model_name: str) -> int:
    """Returns the default image height for the given model_name."""

    return 1024


def get_default_image_width(model_name: str) -> int:
    """Returns the default image width for the given model_name."""

    return 1024


def get_default_num_inference_steps(model_name: str) -> int:
    """Returns the default num_inference_steps for the given model_name."""

    return 4


def generate_images(
    endpoint_name,
    prompt="",
    num_inference_steps=5,
    image_width=1024,
    image_height=1024,
) -> list[Image.Image]:
    if not endpoint_name:
        raise gr.Error("Select (or deploy) a model first.")

    instances = [{"text": prompt}]
    parameters = {
        "height": image_height,
        "width": image_width,
        "num_inference_steps": num_inference_steps,
    }

    response = get_endpoint(endpoint_name).predict(
        instances=instances, parameters=parameters
    )
    images = [
        common_util.base64_to_image(prediction.get("output"))
        for prediction in response.predictions
    ]
    return images


def update_default_parameters(model_name: str):
    """Updates the default inference parameters based on the selected model."""
    return {
        num_inference_steps: gr.update(
            value=get_default_num_inference_steps(model_name)
        ),
        image_width: gr.update(value=get_default_image_width(model_name)),
        image_height: gr.update(value=get_default_image_height(model_name)),
    }


tip_text = r"""
1. Select a previous deployed Flux model.
2. If you don't have any deployed model, deploy a new model. The deployment takes ~20 minutes. You can check the progress at [Vertex Online Prediction](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints). After the model deployment is complete, restart the playground in Colab to see the updated endpoint list.
3. In the "Inference" section, fill the prompt and parameters, then click "Generate" to generate image from text prompt.
"""

css = """
.gradio-container {
  width: 90% !important
}
"""
with gr.Blocks(
    css=css, theme=gr.themes.Default(primary_hue="orange", secondary_hue="blue")
) as demo:
    gr.Markdown("# Model Garden Playground for Flux")

    with gr.Accordion("How To Use", open=True):
        tip = gr.Markdown(tip_text)

    gr.Markdown("## Select or deploy model")
    gr.Markdown("### Select a previously deployed model")
    endpoint_name = gr.Dropdown(
        label="Select a model previously deployed on Vertex",
        choices=list_endpoints(),
        value=None,
    )
    gr.Markdown("### Deploy a new model to Vertex")
    model_id = gr.Dropdown(
        label="Select a model to deploy",
        choices=[
            "black-forest-labs/FLUX.1-schnell",
        ],
        value=None,
    )
    with gr.Row(equal_height=True):
        with gr.Column(scale=7):
            accelerator_type = gr.Dropdown(
                label="Select accelerator type for deployment",
                choices=[
                    "NVIDIA_TESLA_A100",
                    "NVIDIA_A100_80GB",
                ],
                value=None,
            )
        with gr.Column(scale=1):
            deploy_model_button = gr.Button(
                "Deploy", scale=1, variant="primary", min_width=10
            )

    gr.Markdown("## Inference")
    with gr.Row(equal_height=True):
        with gr.Column(scale=1):
            prompt = gr.Textbox(label="Prompt", lines=3)
            gr.Markdown("### Parameters")
            image_width = gr.Slider(
                label="Image width", value=1024, step=128, minimum=512, maximum=1024
            )
            image_height = gr.Slider(
                label="Image height", value=1024, step=128, minimum=512, maximum=1024
            )
            num_inference_steps = gr.Slider(
                label="Inference steps", value=4, step=1, minimum=1, maximum=25
            )
            generate_button = gr.Button("Generate", variant="primary")

        with gr.Column(scale=4):
            with gr.Row(equal_height=True):
                with gr.Column(scale=3):
                    image_output = gr.Gallery(
                        label="Generated Images",
                        rows=1,
                        height=715,
                        preview=True,
                    )

    endpoint_name.change(
        update_default_parameters,
        endpoint_name,
        [
            num_inference_steps,
            image_width,
            image_height,
        ],
    )

    deploy_model_button.click(
        deploy_model,
        inputs=[model_id, accelerator_type],
        outputs=[],
    )

    generate_button.click(
        generate_images,
        inputs=[
            endpoint_name,
            prompt,
            num_inference_steps,
            image_width,
            image_height,
        ],
        outputs=image_output,
    )

show_debug_logs = True  # @param {type: "boolean"}
demo.queue()
demo.launch(share=True, inline=False, debug=show_debug_logs, show_error=True)